<a href="https://colab.research.google.com/github/SERGEYDJUM/believex/blob/main/KNN_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install skl2onnx onnxruntime

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

In [56]:
df = pd.read_csv('working.csv')

In [57]:
df_male_adults = df[(df['sex'] == 0) & (df['age'] >= 18)].dropna(subset=['LF_l'])
df_male_child = df[(df['sex'] == 0) & (df['age'] < 18)].dropna(subset=['LF_l'])
df_female_adults = df[(df['sex'] == 1) & (df['age'] >= 18)].dropna(subset=['LF_l'])
df_female_child = df[(df['sex'] == 1) & (df['age'] < 18)].dropna(subset=['LF_l'])

In [61]:
def prepare_data(df, test_size=0.2, random_state=100):
    X = df[['LF_b', 'HF_b']]
    y = df['LF_l']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

In [62]:
X_train_male_adults, X_test_male_adults, y_train_male_adults, y_test_male_adults = prepare_data(df_male_adults, test_size=0.2, random_state=100)
X_train_male_child, X_test_male_child, y_train_male_child, y_test_male_child = prepare_data(df_male_child, test_size=0.2, random_state=100)
X_train_female_adults, X_test_female_adults, y_train_female_adults, y_test_female_adults = prepare_data(df_female_adults, test_size=0.2, random_state=100)
X_train_female_child, X_test_female_child, y_train_female_child, y_test_female_child = prepare_data(df_female_child, test_size=0.2, random_state=42)

In [115]:
# Обучение моделей KNN для каждой группы
knn_male_adults = KNeighborsRegressor(n_neighbors=5, metric='manhattan')
knn_male_adults.fit(X_train_male_adults, y_train_male_adults)

knn_male_child = KNeighborsRegressor(n_neighbors=5, metric='euclidean', weights='uniform')
knn_male_child.fit(X_train_male_child, y_train_male_child)

knn_female_adults = KNeighborsRegressor(n_neighbors=3, weights='distance')
knn_female_adults.fit(X_train_female_adults, y_train_female_adults)

knn_female_child = KNeighborsRegressor(n_neighbors=3, weights='distance', metric='manhattan')
knn_female_child.fit(X_train_female_child, y_train_female_child)

KNeighborsRegressor(metric='manhattan', n_neighbors=3, weights='distance')

In [116]:
def convert_to_onnx(model, model_name):
    initial_type = [('float_input', FloatTensorType([-1, 2]))]
    onnx_model = convert_sklearn(model, initial_types=initial_type)
    with open(model_name, "wb") as f:
        f.write(onnx_model.SerializeToString())

In [117]:
convert_to_onnx(knn_male_adults, "knn_male_adults.onnx")
convert_to_onnx(knn_male_child, "knn_male_child.onnx")
convert_to_onnx(knn_female_adults, "knn_female_adults.onnx")
convert_to_onnx(knn_female_child, "knn_female_child.onnx")

In [118]:
def predict_with_onnx(model_path, X_test):
    sess = rt.InferenceSession(model_path)
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    # Подаем входные данные в виде словаря
    pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]
    return pred_onx

In [119]:
y_pred_male_adults = predict_with_onnx("knn_male_adults.onnx", X_test_male_adults)
y_pred_male_child = predict_with_onnx("knn_male_child.onnx", X_test_male_child)
y_pred_female_adults = predict_with_onnx("knn_female_adults.onnx", X_test_female_adults)
y_pred_female_child = predict_with_onnx("knn_female_child.onnx", X_test_female_child)

In [120]:
mean_male_adults = df_male_adults['LF_l'].mean()
mean_male_child = df_male_child['LF_l'].mean()
mean_female_adults = df_female_adults['LF_l'].mean()
mean_female_child = df_female_child['LF_l'].mean()
print(f'Mean for men: {mean_male_adults}')
print(f'Mean for boys: {mean_male_child}')
print(f'Mean for women: {mean_female_adults}')
print(f'Mean for girls: {mean_female_child}')

Mean for men: 24.94814814814815
Mean for boys: 25.574782608695653
Mean for women: 18.05
Mean for girls: 28.276363636363637


In [121]:
MSE_male_adults = mean_squared_error(y_test_male_adults, y_pred_male_adults)
print(f'MSE for men: {MSE_male_adults}')
MSE_male_child = mean_squared_error(y_test_male_child, y_pred_male_child)
print(f'MSE for boys: {MSE_male_child}')
MSE_female_adults = mean_squared_error(y_test_female_adults, y_pred_female_adults)
print(f'MSE for women: {MSE_female_adults}')
MSE_female_child = mean_squared_error(y_test_female_child, y_pred_female_child)
print(f'MSE for girls: {MSE_female_child}')

MSE for men: 19.234935362245842
MSE for boys: 46.13882551899535
MSE for women: 91.91139725155331
MSE for girls: 27.313328959147725


In [122]:
MAE_male_adults = mean_absolute_error(y_test_male_adults, y_pred_male_adults)
print(f'MAE for men: {MAE_male_adults}')
MAE_male_child = mean_absolute_error(y_test_male_child, y_pred_male_child)
print(f'MAE for boys: {MAE_male_child}')
MAE_female_adults = mean_absolute_error(y_test_female_adults, y_pred_female_adults)
print(f'MAE for women: {MAE_female_adults}')
MAE_female_child = mean_absolute_error(y_test_female_child, y_pred_female_child)
print(f'MAE for girls: {MAE_female_child}')

MAE for men: 2.826666386922201
MAE for boys: 5.299199829101562
MAE for women: 8.125000476837158
MAE for girls: 4.466666412353516
